In [1]:
import json
import string
import numpy as np

from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence

In [2]:
ds_path = '/home/tim/repos/dynamic-topic-modeling/data/topic_extraction/llama/llama3-chat-prompt-topic-3kw-ami-fixed.json'
#ds_path = '/home/tim/repos/dynamic-topic-modeling/data/superseg/superseg_test_small.json'
#ds_path = '/home/tim/repos/dynamic-topic-modeling/data/topic_extraction/mistral/mistral-chat-prompt-topic-5kw-super.json'
with open(ds_path, 'r') as f:
    keywords = json.load(f)['data']

#labels_path = '/home/tim/repos/dynamic-topic-modeling/data/superseg/superseg_test_small.json'
labels_path = '/home/tim/repos/dynamic-topic-modeling/data/qmsum/ami_test.json'
with open(labels_path, 'r') as f:
    texts = json.load(f)['data']

In [3]:
coh_metrics = []
exclude = set(string.punctuation)

for idx, i in enumerate(keywords):
    current_vocab = []
    cur_sent = [word.lower() for word in texts[idx]['text'].split() if word not in exclude]
    if len(cur_sent) != 0:
        current_vocab.append(cur_sent)
        
    i['topics'] = [[word.lower() for word in i['key_words'] if word != '']]
    if len(i['topics'][0]) == 0:
        coherence_score = 0
    else:
        try:
            coh_metric = Coherence(topk=3, measure='c_npmi', texts=current_vocab)
            coherence_score = coh_metric.score(i)
            if str(coherence_score) == 'nan':
                coherence_score = 0
        except:
            coherence_score = 0
            
    coh_metrics.append(coherence_score)

/home/tim/repos/dynamic-topic-modeling/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tim/repos/dynamic-topic-modeling/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
metrics = []
exclude = set(string.punctuation)
topics_list = {'topics': []}
current_vocab = []

for idx, i in enumerate(keywords):   
    cur_sent = [word.lower() for word in texts[idx]['text'].split() if word not in exclude]
    if len(cur_sent) != 0:
        current_vocab.append(cur_sent)
    
    kw = [word.lower() for word in i['key_words'] if word != '']
    if len(kw) != 0:
        topics_list['topics'].append(kw)
   
td_metric = TopicDiversity(topk=3) 
topic_diversity_score = td_metric.score(topics_list)        

In [5]:
topic_diversity_score, np.mean(coh_metrics)

(0.1871435977190254, -0.3429796104133749)